## 모델 출력 확인하기
1. 모델 학습/평가 관리용 클래스인 ModelManager의 인스턴스를 생성합니다.
2. 학습/평가용으로 불러올 전처리 데이터의 구조는 BaseDataset 클래스로 구현됩니다.
3. BaseDataset은 torch.utils.data.Dataset 클래스를 상속받는데, 이를 통해 전처리 데이터를 DataLoader로 관리할 수 있게 됩니다.
4. DataLoader는 BaseDataset에 구현된 데이터 구조를 batch data 형태로 바꾸기 위해 데이터의 모든 요소에 차원을 하나 추가합니다.<br/>
   -> 상세: BaseDataset은 인스턴스 생성 시(즉 __init__함수에서) list[dict[str, Tensor]] 형태의 데이터를 생성하고 self.behaviors_parsed에 저장합니다.<br/>
   즉 데이터를 하나 뽑으면 dict[str, Tensor] 형태의 구조를 갖습니다.<br/>
   그런데 mini batch 학습을 위해서는 여러개의 데이터를 하나로 묶어서 batch data를 생성해야 하고, 이 기능을 DataLoader로 수행합니다.<br/>
   DataLoader에서는 여러 데이터를 하나로 묶어 dict[str, list[Tensor]] 형태로 변경합니다.<br/>
   list[Tensor]의 길이는 config파일의 batch_size로 정해집니다.<br/>
   여기서 1 epoch의 총 iteration은 behaviors의 총 데이터 수 / batch_size로, 배치 데이터의 총 개수와 같습니다.<br/>
5. 모든 모델 클래스가 상속 받는 pl.LightningModule의 구현 방식으로 인해, 모델의 인스턴스 자체를 함수처럼 사용하면 해당 클래스에 구현된 forward() 함수가 실행됩니다.
6. forward 함수는 학습 시 사용하는 batch data를 받아, behaviors의 사용자 history를 기반으로 해당 사용자의 impression 목록의 click probability를 예측하고 반환합니다.<br/>
   -> 상세: behaviors의 모든 데이터는 크게 유저의 history, 해당 유저의 impressions 데이터로 구성됩니다.<br/>
   여기서 history는 해당 유저가 과거에 열람한 뉴스 목록, impressions는 이러한 history를 가진 유저에게 특정 시점에 화면에 노출된 뉴스 목록입니다.<br/>
   여기서 impressions에는 유저가 해당 뉴스를 클릭했는지(1), 하지 않았는지(0)가 1과 0으로 라벨링 되어있습니다.<br/>
   즉 모델이 history만으로 impressions의 모든 뉴스 목록에 대해 해당 history를 가진 유저의 클릭 가능성을 예측하고, 라벨과 비교하거나 순위를 매겨보면 해당 모델이 추천을 얼마나 정확하게 하는지를 계산할 수 있습니다. 
7. 여기서 반환 형태는 Tensor인데, 내부 데이터는 list[list[float]] 형태입니다. 즉 입력한 모든 batch data에 대한 예측 결과가 반환되는 것입니다.<br/>
   -> 상세: 예를 들어 batch_size가 2라면, 각 배치마다 behaviors의 데이터가 2개씩 포함될 것입니다.<br/>
    따라서 예측해야할 유저와 impression 쌍도 두개이므로, 반환하는 결과 데이터도 2개입니다.

In [2]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

## 1. ckpt 파일로 모델 불러오기

In [3]:
import os
from os import path
import sys

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)

from utils.model_manager import ModelManager
from utils.base_manager import ManagerArgs

In [4]:
args = ManagerArgs(
    config_path = path.join(PROJECT_DIR, "config/model/nrms/exp_demo1.yaml"),
    test_ckpt_path = path.join(PROJECT_DIR, "logs/lightning_logs/checkpoints/nrms/exp_demo1/epoch=24-val_auc_epoch=0.6996.ckpt")
)

model_manager = ModelManager(PROJECT_DIR, args, "test")

Seed set to 1234
100%|██████████| 7538/7538 [00:07<00:00, 1021.53it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## 2. 데이터 확인용

In [7]:
"""
모든 데이터의 첫 번째 차원의 shape 값은 batch size입니다.
즉 batch data 안에 실제로 어떤 데이터가 저장되어있는지 알아보기 위해 출력해볼 때
해당 값은 별로 의미가 없습니다. 

예를 들어 h_title의 shape 출력 결과의 각 숫자는 다음과 같은 의미를 지닙니다.
(batch_size, config파일에 설정한 max_history 값, 전처리 과정에서 설정한 max_title 값 = 제목의 최대 토큰 개수)

c_abstract은 다음과 같습니다.
(batch_size, 해당 impressions 데이터에 포함된 뉴스 개수, 전처리 과정에서 설정한 max_abstract 값 = 본문 요약의 최대 토큰 개수)
"""

batch_index = 0
batch_data = model_manager.get_batch_from_dataloader(batch_index)
model_manager.show_batch_struct(batch_data)

<class 'dict'>
{
	user:	type=Tensor, shape=(1,), inner_type=int
	h_idxs:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_title:	type=Tensor, shape=(1, 50, 20), inner_type=list[list[int]]
	h_abstract:	type=Tensor, shape=(1, 50, 50), inner_type=list[list[int]]
	h_category:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_subcategory:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_vader_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	h_bert_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	history_length:	type=Tensor, shape=(1,), inner_type=int
	c_idxs:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_title:	type=Tensor, shape=(1, 28, 20), inner_type=list[list[int]]
	c_abstract:	type=Tensor, shape=(1, 28, 50), inner_type=list[list[int]]
	c_category:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_subcategory:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_vader_sentiment:	type=Tensor, shape=(1, 28), inner_type=list[float]
	c_bert_sentime

## 3. 추천 순위 top N 뉴스의 정보 출력하기

In [ ]:
batch_index = 6
sample_num = 50
topN = 5

model_manager.show_output_by_index(batch_index)
print("\n\n")
model_manager.show_history(batch_index, sample_num)
print("\n\n")
model_manager.show_topN_result(batch_index, topN)

Rank    Score    Label  index 
--------------------------------
1      28.63771    0      27  
2      25.09192    1      5   
3      20.50810    0      23  
4      18.70691    0      15  
5      14.14925    0      29  
6      12.78966    0      19  
7      9.44058     0      14  
8      8.65007     0      1   
9      6.99425     0      6   
10     5.17248     0      10  
11     4.89551     0      12  
12     3.74166     0      21  
13     3.70619     0      33  
14     1.14551     0      30  
15     0.00914     0      28  
16     -0.65226    0      22  
17     -0.73956    0      24  
18     -1.03457    0      0   
19     -1.04225    0      8   
20     -1.28145    0      36  
21     -3.42915    0      25  
22     -6.19252    0      17  
23     -6.34231    0      3   
24     -9.15500    0      4   
25     -9.55248    0      26  
26    -10.15843    0      20  
27    -11.71873    0      34  
28    -14.87641    0      16  
29    -16.91944    0      18  
30    -19.89777    0      7   
31    